## This notebook is mostly based on the New York Neighborhood lab notebook:
#### DP0701EN-3-3-2-Neighborhoods-New-York-py-v1.0.ipynb

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [173]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

For convenience, I uploaded the file to github, so I only need to run a 'wget' command and access the data. So let's go ahead and do that.

In [209]:
!wget -q -O 'toronto.json' https://raw.githubusercontent.com/ferny65/github-example/Capstone-Project/toronto.json

print('Data downloaded!')

Data downloaded!


#### Load and explore the data

Next, let's load the data.

In [210]:
with open('toronto.json') as json_data:
    toronto_data = json.load(json_data)

Let's take a look at the data.

In [211]:
toronto_data['Borough']

{u'0': u'Scarborough',
 u'1': u'Scarborough',
 u'10': u'Scarborough',
 u'100': u'Etobicoke',
 u'101': u'Etobicoke',
 u'102': u'Etobicoke',
 u'11': u'Scarborough',
 u'12': u'Scarborough',
 u'13': u'Scarborough',
 u'14': u'Scarborough',
 u'15': u'Scarborough',
 u'16': u'Scarborough',
 u'17': u'North York',
 u'18': u'North York',
 u'19': u'North York',
 u'2': u'Scarborough',
 u'20': u'North York',
 u'21': u'North York',
 u'22': u'North York',
 u'23': u'North York',
 u'24': u'North York',
 u'25': u'North York',
 u'26': u'North York',
 u'27': u'North York',
 u'28': u'North York',
 u'29': u'North York',
 u'3': u'Scarborough',
 u'30': u'North York',
 u'31': u'North York',
 u'32': u'North York',
 u'33': u'North York',
 u'34': u'North York',
 u'35': u'East York',
 u'36': u'East York',
 u'37': u'East Toronto',
 u'38': u'East York',
 u'39': u'East York',
 u'4': u'Scarborough',
 u'40': u'East York',
 u'41': u'East Toronto',
 u'42': u'East Toronto',
 u'43': u'East Toronto',
 u'44': u'Central Toront

This is not necesary, but for convenience...

In [212]:
neighborhoods_data = toronto_data

## 2. Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe.

In [213]:
# define the dataframe columns
column_names = ['Postalcode','Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

Then let's  assign each dictionary to a list and create the dataframe.

In [214]:
#creating the columns from json data
borough = list(toronto_data['Borough'].values()) 
postalcode =   list(toronto_data['Postalcode'].values())  
neighborhood = list(toronto_data['Neighbourhood'].values())
latitude = list(toronto_data['Latitude_x'].values())
longitude = list(toronto_data['Longitude_x'].values())    
# instantiate the dataframe
d = {'Postalcode':postalcode,
     'Borough': borough,
     'Neighborhood': neighbourhood,
     'Latitude': latitude,
     'Longitude': longitude}
neighborhoods = pd.DataFrame(d, columns=column_names)

Quickly examine the resulting dataframe.

In [215]:
neighborhoods.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M2R,North York,Willowdale West,43.782736,-79.442259
1,M3A,North York,Parkwoods,43.753259,-79.329656
2,M3B,North York,Don Mills North,43.745906,-79.352188
3,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923
4,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714


Printing number of boroughs and neighborhoods.

In [216]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>to_explorer</em>, as shown below.

In [217]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Filtering dataframe by Boroughs containing the word Toronto

### Printing number of boroughs containing "Toronto"

In [218]:
print('{} boroughs contains the word Toronto'.format(toronto_neighborhoods.shape[0]))

38 boroughs contains the word Toronto


In [219]:
toronto_neighborhoods= neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_neighborhoods

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
1,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
7,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
8,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
9,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160


## 3. Create map of Boroughs containing "Toronto" using latitude and longitude values

Create a map of Toronto with neighborhoods superimposed on top.

In [220]:
map_boroughs_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'], toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_boroughs_toronto)  
    
map_boroughs_toronto

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

## 4. Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [221]:
CLIENT_ID = 'XDWICBLRXVDJIO500WN34H4A2ALB22WUGO0CGTJDP21SPGWL' # your Foursquare ID
CLIENT_SECRET = 'HX1DQVKYP1UOLT0QZB0E53L1TBQY34FRFUROUO1D1MMZDA4A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XDWICBLRXVDJIO500WN34H4A2ALB22WUGO0CGTJDP21SPGWL
CLIENT_SECRET:HX1DQVKYP1UOLT0QZB0E53L1TBQY34FRFUROUO1D1MMZDA4A


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [222]:
toronto_neighborhoods.loc[0,'Neighborhood']

u'Harbourfront East, Toronto Islands, Union Station'

Get the neighborhood's latitude and longitude values.

In [223]:
neighborhood_latitude = toronto_neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront East, Toronto Islands, Union Station are 43.6408157, -79.3817523.


#### Now, let's get the top 100 venues that are in Harbourfront East, Toronto Islands, Union Stationwithin a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [224]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=XDWICBLRXVDJIO500WN34H4A2ALB22WUGO0CGTJDP21SPGWL&client_secret=HX1DQVKYP1UOLT0QZB0E53L1TBQY34FRFUROUO1D1MMZDA4A&v=20180605&ll=43.6408157,-79.3817523&radius=500&limit=100'

Send the GET request and examine the resutls

In [225]:
results = requests.get(url).json()
results

{u'meta': {u'code': 200, u'requestId': u'5d850867a4b51b0038ae56cc'},
 u'response': {u'groups': [{u'items': [{u'reasons': {u'count': 0,
       u'items': [{u'reasonName': u'globalInteractionReason',
         u'summary': u'This spot is popular',
         u'type': u'general'}]},
      u'referralId': u'e-0-4bfaa3494a67c928d08528cf-0',
      u'venue': {u'categories': [{u'icon': {u'prefix': u'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
          u'suffix': u'.png'},
         u'id': u'4f2a25ac4b909258e854f55f',
         u'name': u'Neighborhood',
         u'pluralName': u'Neighborhoods',
         u'primary': True,
         u'shortName': u'Neighborhood'}],
       u'id': u'4bfaa3494a67c928d08528cf',
       u'location': {u'cc': u'CA',
        u'city': u'Toronto',
        u'country': u'Canada',
        u'distance': 167,
        u'formattedAddress': [u'Toronto ON', u'Canada'],
        u'labeledLatLngs': [{u'label': u'display',
          u'lat': 43.639525632239106,
         

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [226]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Harbourfront,Neighborhood,43.639526,-79.380688
1,Roundhouse Park,Park,43.641745,-79.384279
2,iQ Food Co,Salad Place,43.642851,-79.382081
3,BeaverTails,Dessert Shop,43.639736,-79.380068
4,Aroma Espresso Bar,Café,43.642321,-79.383749


And how many venues were returned by Foursquare?

In [227]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<a id='item2'></a>

## 5. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [228]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [229]:
toronto_venues = getNearbyVenues(names=toronto_neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



Harbourfront East, Toronto Islands, Union Station
Adelaide, King, Richmond
St. James Town
Ryerson, Garden District
Central Bay Street
Berczy Park
Cabbagetown, St. James Town
Rosedale
Harbourfront, Regent Park
Church and Wellesley
High Park, The Junction South
Parkdale, Roncesvalles
Business Reply Mail Processing Centre 969 Eastern
Runnymede, Swansea
The Beaches
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
North Toronto West
Davisville
Lawrence Park
Davisville North
The Beaches West, India Bazaar
Studio District
The Danforth West, Riverdale
Little Portugal, Trinity
Dovercourt Vill

#### Let's check the size of the resulting dataframe

In [230]:
print(toronto_venues.shape)
toronto_venues.head()

(1041, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront East, Toronto Islands, Union Station",43.782736,-79.442259,Shoppers Drug Mart,43.784847,-79.446028,Pharmacy
1,"Harbourfront East, Toronto Islands, Union Station",43.782736,-79.442259,Tov-Li,43.784214,-79.446098,Pizza Place
2,"Harbourfront East, Toronto Islands, Union Station",43.782736,-79.442259,Tim Hortons,43.780940,-79.444231,Coffee Shop
3,"Harbourfront East, Toronto Islands, Union Station",43.782736,-79.442259,Dollarama,43.784670,-79.446670,Discount Store
4,"Harbourfront East, Toronto Islands, Union Station",43.782736,-79.442259,Hartman's,43.784312,-79.446213,Butcher


Let's check how many venues were returned for each neighborhood

In [231]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",3,3,3,3,3,3
"Brockton, Exhibition Place, Parkdale Village",9,9,9,9,9,9
Business Reply Mail Processing Centre 969 Eastern,3,3,3,3,3,3
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",91,91,91,91,91,91
"Cabbagetown, St. James Town",34,34,34,34,34,34
"Chinatown, Grange Park, Kensington Market",49,49,49,49,49,49
Christie,4,4,4,4,4,4
Church and Wellesley,6,6,6,6,6,6
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [232]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 213 uniques categories.


<a id='item3'></a>

## 6. Analyze Each Neighborhood

In [233]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="").drop('Neighborhood', axis=1)

# add neighborhood column back to dataframe
toronto_onehot.insert(0, 'Neighborhood', toronto_venues['Neighborhood'])  

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Harbourfront East, Toronto Islands, Union Station",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront East, Toronto Islands, Union Station",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront East, Toronto Islands, Unio

And let's examine the new dataframe size.

In [234]:
toronto_onehot.shape

(1041, 213)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [235]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.00000,0.333333,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,

#### Let's confirm the new size

In [236]:
toronto_grouped.shape

(36, 213)

#### Let's print each neighborhood along with the top 5 most common venues

In [237]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
               venue  freq
0               Pool  0.33
1  Food & Drink Shop  0.33
2               Park  0.33
3  Afghan Restaurant  0.00
4             Museum  0.00


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0   Grocery Store  0.22
1  Sandwich Place  0.11
2     Pizza Place  0.11
3        Pharmacy  0.11
4     Coffee Shop  0.11


----Business Reply Mail Processing Centre 969 Eastern----
                 venue  freq
0                Motel  0.67
1  American Restaurant  0.33
2          Music Venue  0.00
3        Metro Station  0.00
4   Mexican Restaurant  0.00


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                 venue  freq
0          Coffee Shop  0.09
1  Japanese Restaurant  0.07
2              Gay Bar  0.05
3     Sushi Restaurant  0.04
4           Restaurant  0.03


----Cabbagetown, St. James Town----
              venue  freq
0  Ramen Restaura

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [238]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [275]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Pool,Food & Drink Shop,Park,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,"Brockton, Exhibition Place, Parkdale Village",Grocery Store,Pharmacy,Fried Chicken Joint,Pizza Place,Sandwich Place,Beer Store,Fast Food Restaurant,Coffee Shop,Cosmetics Shop,Donut Shop
2,Business Reply Mail Processing Centre 969 Eastern,Motel,American Restaurant,Yoga Studio,Dog Run,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
3,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Gastropub,Men's Store,Mediterranean Restaurant,Hotel,Gym
4,"Cabbagetown, St. James Town",Coffee Shop,Ramen Restaurant,Café,Japanese Restaurant,Pizza Place,Sandwich Place,Lounge,Fast Food Restaurant,Juice Bar,Bubble Tea Shop


<a id='item4'></a>

## 7. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [271]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 4, 2, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [276]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_neighborhoods
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood').dropna()
toronto_merged = toronto_merged.reset_index(drop=True)
toronto_merged # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,0.0,Pharmacy,Coffee Shop,Butcher,Pizza Place,Discount Store,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,1.0,Pool,Food & Drink Shop,Park,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,Japanese Restaurant,Caribbean Restaurant,Gym / Fitness Center,Café,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
3,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0.0,Gym,Beer Store,Asian Restaurant,Coffee Shop,Concert Hall,Chinese Restaurant,Sandwich Place,Restaurant,Bike Shop,Sporting Goods Shop
4,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0.0,Coffee Shop,Ramen Restaurant,Café,Japanese Restaurant,Pizza Place,Sandwich Place,Lounge,Fast Food Restaurant,Juice Bar,Bubble Tea Shop
5,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1.0,Convenience Store,Park,Bank,Yoga Studio,Dog Run,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
6,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0.0,Coffee Shop,Pizza Place,Middle Eastern Restaurant,Restaurant,Supermarket,Sushi Restaurant,Deli / Bodega,Bank,Fried Chicken Joint,Frozen Yogurt Shop
7,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0.0,Massage Studio,Bar,Coffee Shop,Falafel Restaurant,Caribbean Restaurant,Furniture / Home Store,Festival,Fast Food Restaurant,Farmers Market,Event Space
8,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,4.0,Fast Food Restaurant,Yoga Studio,Fish Market,Festival,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
9,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,0.0,Bank,Athletics & Sports,Bakery,Fried Chicken Joint,Thai Restaurant,Caribbean Restaurant,Lounge,Hakka Restaurant,Falafel Restaurant,Event Space


Finally, let's visualize the resulting clusters

In [277]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

#### Cluster 1

In [278]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0.0,Pharmacy,Coffee Shop,Butcher,Pizza Place,Discount Store,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
2,Downtown Toronto,0.0,Japanese Restaurant,Caribbean Restaurant,Gym / Fitness Center,Café,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
3,Downtown Toronto,0.0,Gym,Beer Store,Asian Restaurant,Coffee Shop,Concert Hall,Chinese Restaurant,Sandwich Place,Restaurant,Bike Shop,Sporting Goods Shop
4,Downtown Toronto,0.0,Coffee Shop,Ramen Restaurant,Café,Japanese Restaurant,Pizza Place,Sandwich Place,Lounge,Fast Food Restaurant,Juice Bar,Bubble Tea Shop
6,Downtown Toronto,0.0,Coffee Shop,Pizza Place,Middle Eastern Restaurant,Restaurant,Supermarket,Sushi Restaurant,Deli / Bodega,Bank,Fried Chicken Joint,Frozen Yogurt Shop
7,Downtown Toronto,0.0,Massage Studio,Bar,Coffee Shop,Falafel Restaurant,Caribbean Restaurant,Furniture / Home Store,Festival,Fast Food Restaurant,Farmers Market,Event Space
9,West Toronto,0.0,Bank,Athletics & Sports,Bakery,Fried Chicken Joint,Thai Restaurant,Caribbean Restaurant,Lounge,Hakka Restaurant,Falafel Restaurant,Event Space
11,West Toronto,0.0,Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Brewery,Fried Chicken Joint,Music Venue,Bakery,Bar
12,East Toronto,0.0,Coffee Shop,Café,Steakhouse,Bar,Cosmetics Shop,American Restaurant,Restaurant,Burger Joint,Hotel,Thai Restaurant
13,Downtown Toronto,0.0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Cocktail Bar,Bakery,Cosmetics Shop,Clothing Store,Breakfast Spot


#### Cluster 2

In [279]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,1.0,Pool,Food & Drink Shop,Park,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
5,Downtown Toronto,1.0,Convenience Store,Park,Bank,Yoga Studio,Dog Run,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
18,Downtown Toronto,1.0,Park,Playground,Building,Trail,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


#### Cluster 3

In [280]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Central Toronto,2.0,Convenience Store,Grocery Store,Bus Line,Caribbean Restaurant,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


#### Cluster 4

In [281]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,East Toronto,3.0,Motel,American Restaurant,Yoga Studio,Dog Run,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


#### Cluster 5

In [282]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,West Toronto,4.0,Fast Food Restaurant,Yoga Studio,Fish Market,Festival,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).